In [ ]:
from datasets import load_dataset
import pandas as pd
from utils import  *


In [ ]:
rste = load_dataset("razhan/rste", split="train")

In [ ]:
df_rste = rste.to_pandas()
df = df_rste
pd.set_option('display.max_colwidth', None)

In [ ]:
df['clean_text'] = df['text'].apply(process_text)

# df['contains_non_kurdish'] = df["text"].apply(contains_non_kurdish_characters)
# print(df['contains_non_kurdish'].sum())
# df[df['contains_non_kurdish'] == True]

In [ ]:
df['clean_text'] = df['clean_text'].apply(keep_kurdish_characters)

In [ ]:
# df[df['contains_non_kurdish'] == False]['clean_text'].to_csv("data/data.ckb.txt", index=False, header=False)
df['clean_text'].to_csv("data/data.ckb.txt", index=False, header=False)

In [ ]:
df[df['text'].str.contains('ھ')]
indices_with_substring = df[df['text'].str.contains('ھ')].index
# print(indices_with_substring)
df.loc[indices_with_substring]

In [ ]:
all_text = ''.join(df["text"])

unique_characters = set(all_text)

print("Unique characters:", unique_characters)
print("Number of unique characters:", len(unique_characters))

In [ ]:
all_text = ''.join(df["clean_text"])
# all_text = ''.join(df[df['contains_non_kurdish'] == False]['clean_text'])
unique_characters = set(all_text)
unique_punctuations = extract_punctuation(all_text)
print("Unique characters:", unique_characters)
print("Number of unique characters:", len(unique_characters))
print("Unique punctuations:", unique_punctuations)

In [ ]:
df['contains_non_kurdish'] = df["text"].apply(contains_non_kurdish_characters)

In [ ]:
len(unique_punctuations)

In [ ]:
# df = pd.read_csv("asosoft_test_punc.csv")
# df['summary'] = df['summary'].apply(process_text)
# df.to_csv("asosoft_test_clean.csv", index=False)

In [ ]:
df

In [ ]:
oscar_dataset = load_dataset("oscar-corpus/OSCAR-2301", language="ckb", split='train', token=True)
wiki_dataset = load_dataset("wikipedia", language="ckb", date="20231120", split='train', beam_runner='DirectRunner')

df_oscar = oscar_dataset.to_pandas()
df_wiki = wiki_dataset.to_pandas()
df = pd.concat([df_oscar, df_wiki], ignore_index=True)


In [ ]:
df

In [ ]:
df["text"] = df["text"].apply(process_text)

In [ ]:
# text = df["text"].str.cat(sep="\n")

In [ ]:
df['clean_text'] = df['text'].apply(keep_kurdish_characters)

In [ ]:
df["clean_text"] = df["clean_text"].apply(process_text)

In [ ]:
df['contains_non_kurdish'] = df["clean_text"].apply(contains_non_kurdish_characters)
print(df['contains_non_kurdish'].sum())
# df[df['contains_non_kurdish'] == True].iloc[0]['clean_text']

In [ ]:
df['repeated_ngram'] = df['clean_text'].apply(lambda x: contains_repeated_ngram(x, 10))
print(df['repeated_ngram'].sum())

In [ ]:
# drop rows where repeated_ngram are True
df = df[df['repeated_ngram'] == False]


In [ ]:
df[df['repeated_ngram'] == True].iloc[0]['clean_text']

In [ ]:
all_text = "".join(df["clean_text"])
# all_text = ''.join(df[df['contains_non_kurdish'] == False]['clean_text'])
unique_characters = set(all_text)
unique_punctuations = extract_punctuation(all_text)
print("Unique characters:", unique_characters)
print("Number of unique characters:", len(unique_characters))
print("Unique punctuations:", unique_punctuations)

In [ ]:
from nltk.tokenize import sent_tokenize
data = []
for i, row in df.iterrows():
    sentences = tokenizer_ckb.sent_tokenize(row['clean_text'])
    # sentences = row['clean_text'].split('\n')
    # sentences = sent_tokenize(row['clean_text'])
    sentences = [sent_tokenize(s) for s in sentences]
    # flatten list of lists
    sentences = [item for sublist in sentences for item in sublist]
    # split on period and keep the period
    sentences = [s.split('.') for s in sentences]
    sentences = [item for sublist in sentences for item in sublist]

    sentences = [s + '.' for s in sentences]
    data.extend(sentences)
    # if i == 5:
    #     break

In [ ]:
print(len(data))

In [ ]:
# longest line in data
max_line = max(data, key=len)

In [ ]:
len(max_line.split())

In [ ]:
max_line

In [ ]:
# calulate the length of each line in the data and take the average
lengths = [len(line.split()) for line in data]
avg_length = sum(lengths) / len(lengths)
print(avg_length)

In [ ]:
# give me all the lines above 20 words
long_lines = [line for line in data if len(line.split()) > 25]
print(len(long_lines))

In [ ]:
# Write sentences to file
with open("data/oscar_wiki.ckb.txt", "w") as f:
    for sentence in data:
        f.write(sentence + "\n")